<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/notion_expense_sync_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# ✅ BLOCK 0: ติดตั้งไลบรารีที่ต้องใช้
# --------------------------------------------------
# ใช้สำหรับ:
# - เชื่อม Google Sheet (gspread, oauth2client)
# - โหลด .env (python-dotenv)
# - ดึงข้อมูลไปยัง Notion (notion-client)
# --------------------------------------------------

!pip install gspread oauth2client python-dotenv notion-client

In [59]:
# ✅ BLOCK 1: Mount Google Drive
# --------------------------------------------------
# เชื่อม Google Colab กับ Google Drive
# เพื่อเข้าถึงไฟล์เช่น .env หรือ service_account.json
# --------------------------------------------------

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [46]:
# ✅ BLOCK 2: Load .env และดึงค่าตัวแปรสำคัญ
# --------------------------------------------------
# - ใช้สำหรับโหลดค่าที่ตั้งไว้ในไฟล์ .env เช่น:
#   SHEET_ID, NOTION_TOKEN, NOTION_DB_ID
# - ปลอดภัยกว่าการเขียน Token ลงในโค้ดโดยตรง
# --------------------------------------------------

from dotenv import load_dotenv
import os

# 📂 กำหนด path ของ .env ที่เก็บอยู่ใน Google Drive
env_path = "/content/drive/MyDrive/GAS_Automation_Hub/Automation_Keys/Notion_Expense/.env"
load_dotenv(dotenv_path=env_path)

# 🎯 โหลดค่าจาก .env
SHEET_ID = os.getenv("SHEET_ID")
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_DB_ID = os.getenv("NOTION_DB_ID")

# ✅ ตรวจสอบว่าโหลดตัวแปรมาครบไหม
print("SHEET_ID =", SHEET_ID)
print("NOTION_TOKEN =", "✔️ Loaded" if NOTION_TOKEN else "❌ Missing")
print("NOTION_DB_ID =", "✔️ Loaded" if NOTION_DB_ID else "❌ Missing")

SHEET_ID = 1j9GE9KsQnZdUVohsDJ9e97FDqQC-g1-Cv81L1olnHu4
NOTION_TOKEN = ✔️ Loaded
NOTION_DB_ID = ✔️ Loaded


In [47]:
# ✅ BLOCK 3: ตั้ง path และตรวจสอบไฟล์ service_account.json
# --------------------------------------------------
# - ใช้สำหรับยืนยันตัวตนเพื่อให้ Python เข้าถึง Google Sheet ผ่าน API
# - ได้มาจาก Google Cloud Console (ไฟล์ JSON)
# --------------------------------------------------

# 🛠 ตั้ง path ให้ตรงกับที่คุณ find เจอจริง ๆ
service_account_path = "/content/drive/MyDrive/GAS_Automation_Hub/Automation_Keys/Notion_Expense/service_account.json"

# ✅ ตรวจสอบว่าไฟล์นี้อยู่จริงใน Google Drive
import os
print("service_account.json exists ✅:", os.path.exists(service_account_path))

service_account.json exists ✅: True


In [48]:
# ✅ BLOCK 4: เชื่อมต่อ Google Sheet ด้วย gspread
# --------------------------------------------------
# - ใช้ credentials จาก service_account.json เพื่อ authorize
# - เชื่อม Google Sheet ด้วย SHEET_ID ที่ดึงมาจาก .env
# --------------------------------------------------

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 🔐 ระบุ scope ที่ให้สิทธิ์ Google Sheet + Drive
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# 🗝️ โหลด credential จาก service_account.json และ authorize
creds = ServiceAccountCredentials.from_json_keyfile_name(service_account_path, scope)
client = gspread.authorize(creds)

# 📄 เปิด Google Sheet จาก SHEET_ID และเปิด Sheet แรก (.sheet1)
sheet = client.open_by_key(SHEET_ID).sheet1

# ✅ ทดสอบว่าเชื่อมต่อได้จริงไหม
print("🔗 Connected to Google Sheet:", sheet.title)

🔗 Connected to Google Sheet: Sheet1


In [50]:
# ✅ BLOCK 5: ฟังก์ชันบันทึกข้อความลง Google Sheet
# --------------------------------------------------
# - ใช้เมื่อมีข้อความจากผู้ใช้ (เช่นผ่าน LINE หรืออื่น ๆ)
# - จะบันทึก [เวลา, user_id, ข้อความ, Synced?=NO] ลงไปใน Sheet
# --------------------------------------------------

import datetime

def log_message(user_id, message):
    # 📆 บันทึกเวลาปัจจุบันในรูปแบบ ISO (เช่น 2025-06-19T15:10:01)
    now = datetime.datetime.now().isoformat()

    # 📝 เพิ่มแถวใหม่ลง Sheet: [เวลา, user_id, ข้อความ, Synced?]
    sheet.append_row([now, user_id, message, "NO"])

    print(f"✅ Logged: [{now}] {user_id} → {message}")

In [58]:
# ✅ BLOCK 6: Sync Google Sheet → Notion (รองรับหลายหมวด)
from notion_client import Client
notion = Client(auth=NOTION_TOKEN)

# ✅ Mapping หมวดหมู่ → Database ID จาก .env
category_db_map = {
    "ข้าว": os.getenv("NOTION_DB_ID_FOOD"),
    "น้ำ": os.getenv("NOTION_DB_ID_FOOD"),
    "ผัก": os.getenv("NOTION_DB_ID_GROCERIES"),
    "ของใช้": os.getenv("NOTION_DB_ID_SHOPPING"),
    "รถ": os.getenv("NOTION_DB_ID_TRANSPORT"),
    "หนัง": os.getenv("NOTION_DB_ID_ENTERTAINMENT"),
    "ค่าไฟ": os.getenv("NOTION_DB_ID_UTILITIES"),
    "ออกกำลัง": os.getenv("NOTION_DB_ID_HEALTH"),
    "บ้าน": os.getenv("NOTION_DB_ID_HOME"),
    "ออมเงิน": os.getenv("NOTION_DB_ID_SAVINGS"),
    "ดินเนอร์": os.getenv("NOTION_DB_ID_SPECIALDINNER"),
    "ตัดผม": os.getenv("NOTION_DB_ID_HAIRCUT"),
    "เงินสด": os.getenv("NOTION_DB_ID_CASH"),
    "พัฒนา": os.getenv("NOTION_DB_ID_SELFIMPROVEMENT"),
    "บริจาค": os.getenv("NOTION_DB_ID_DONATION"),
}

# ✅ แปลงข้อความเป็น (หมวด, จำนวนเงิน)
def parse_message(text):
    try:
        parts = text.strip().split()
        if len(parts) == 2:
            category = parts[0]
            amount = float(parts[1])
            return category, amount
        elif len(parts) == 1:
            return parts[0], 0.0
        else:
            return None, None
    except:
        return None, None

# ✅ ส่งข้อมูลไปยัง Notion (ตามหมวด)
def push_to_notion(category, amount):
    db_id = category_db_map.get(category)
    if not db_id:
        print(f"❌ ไม่พบ Database สำหรับหมวด '{category}'")
        return

    notion.pages.create(
        parent={"database_id": db_id},
        properties={
            "Name": {"title": [{"text": {"content": category}}]},
            "Amount": {"number": amount}
        }
    )

# ✅ Loop จาก Sheet → ถ้ายังไม่ Synced → ส่งไปยังหมวดนั้นใน Notion
def sync_sheet_to_notion():
    data = sheet.get_all_records()
    for idx, row in enumerate(data):
        if row['Synced?'].strip().upper() != "NO":
            continue

        category, amount = parse_message(row['Message'])
        if not category or amount is None:
            print(f"❌ Format ผิดที่ row {idx+2}: {row['Message']}")
            continue

        try:
            push_to_notion(category, amount)
            sheet.update_cell(idx + 2, 4, "YES")
            print(f"✅ Synced row {idx+2}: {category} {amount}")
        except Exception as e:
            print(f"🚨 Error at row {idx+2}: {e}")
